# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
import json
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'local', 'met', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [10]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63f1ab37ff647a919436eaf7'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1034540',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [11]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [51]:
# Create a dictionary for the new restaurant data
Penang_Flavours = {'BusinessName':'Penang Flavours','AddressLine1': 'Greenwich'}

In [52]:
# Insert the new restaurant into the collection
establishments.insert_one(Penang_Flavours)

In [53]:
# Check that the new restaurant was inserted
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    pprint(result)

{'AddressLine1': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 '_id': ObjectId('63f1d6d9e05ffd0a6e1b1fb6')}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [54]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = establishments.find_one({'BusinessType': 'Restaurant/Cafe/Canteen'})

print(f"The BusinessType {query['BusinessType']} has the BusinessTypeID of {query['BusinessTypeID']}")

The BusinessType Restaurant/Cafe/Canteen has the BusinessTypeID of 1


3. Update the new restaurant with the `BusinessTypeID` you found.

In [55]:
# Update the new restaurant with the correct BusinessTypeID
query = establishments.find_one({'BusinessName': 'Penang Flavours'})
establishments.update_many(query,{'$set':{"BusinessType" : "Restaurant/Cafe/Canteen",\
                                                "BusinessTypeID" : 1}})

In [56]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    pprint(result)

{'AddressLine1': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('63f1d6d9e05ffd0a6e1b1fb6')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [57]:
# Find how many documents have LocalAuthorityName as "Dover"
doverCount = establishments.count_documents({'LocalAuthorityName': 'Dover'})
doverCount

994

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"


In [ ]:
# Check if any remaining documents include Dover


In [ ]:
# Check that other documents remain with 'find_one'


5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude


In [ ]:
# Change the data type from String to Decimal for latitude


In [ ]:
# Check that the coordinates are now numbers
